In [1]:
from pygeoinf.interval import (
Lebesgue, Sobolev, Laplacian,
IntervalDomain, BoundaryConditions,
BesselSobolevInverse, Gradient, Function,
SOLAOperator, InverseLaplacian)
from pygeoinf import EuclideanSpace, LinearOperator
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Set up the corrected configuration for investigating Laplacian accuracy
func_domain = IntervalDomain(0, 1)
bcs = BoundaryConditions('dirichlet')
dim_M = 1  # Use higher resolution for better accuracy

# Create spaces with consistent dimensions
M_underlying = Lebesgue(dim_M, func_domain, basis='fourier')
L = Laplacian(M_underlying, bcs, dofs=100)
M = Sobolev(M_underlying, 1.0, 1.0, L)

D = EuclideanSpace(1)  # Data space
# Create Laplacian operator for testing
kernel = Function(M_underlying, evaluate_callable=lambda x: np.sin(10*np.pi*x) + np.exp(x**2) * np.sin(100 * np.pi * x))
kernel_sobolev = Function(M, evaluate_callable=lambda x: np.sin(10*np.pi*x) + np.exp(x**2) * np.sin(100 * np.pi * x))
G = SOLAOperator(M_underlying, D, kernels=[kernel])
G_sobolev = SOLAOperator(M, D, kernels=[kernel_sobolev])


TypeError: Sobolev.__init__() takes 4 positional arguments but 5 were given

In [ ]:
G.adjoint(D.basis_vector(0)).plot()
G_sobolev.adjoint(D.basis_vector(0)).plot()

NameError: name 'LinearFormSobolev' is not defined